In [ ]:
import pandas as pd
from pathlib import Path
#import py3Dmol
import warnings
from pocket_embed_new import process_pocket,_handle_alt_loc_df,standardize_pdb
# 忽略pandas读取固定宽度文件时的性能警告
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
import os
import torch
import tempfile
import argparse
import pickle
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import io
import warnings

# 忽略pandas读取固定宽度文件时的性能警告
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

# Assuming ESM-3 and other dependencies are installed
from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein, LogitsConfig

print("库导入成功！")

/data/share/liuzhiyuan/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


库导入成功！


In [6]:
# --- 请修改为您的文件路径 ---
# 假设您已将test_set解压到data目录下
pdb_path = Path("/mnt/rna01/liuzhiyuan/zyliu/nai/NExT-Mol/data/cross_docked_new/crossdocked_pocket/CARP_CRYPA_90_419_catalytic_0/3wz8_A_rec_5hct_61p_lig_tt_min_0_pocket10.pdb")

if not pdb_path.exists():
    print(f"错误：文件未找到！请检查路径：{pdb_path}")
else:
    print(f"成功找到文件：{pdb_path.name}")


# --- Improvement 1: Load the model ONCE ---
print("[*] Loading ESM-3 model... (this may take a moment)")
device = torch.device(1)
# Use bfloat16 for faster inference and less memory usage if available
model = ESM3.from_pretrained("esm3-sm-open-v1").to(device).eval()
print("[*] Model loaded successfully.")

# --- Improvement 2: Discover all files using your index.pkl ---

output_path ='/mnt/rna01/liuzhiyuan/zyliu/nai/NExT-Mol/test_scripts'



    
embedding = process_pocket(str(pdb_path), model, device)
if embedding is not None:
        torch.save(embedding, output_path)


print("[*] All pockets processed successfully.")




成功找到文件：3wz8_A_rec_5hct_61p_lig_tt_min_0_pocket10.pdb
[*] Loading ESM-3 model... (this may take a moment)


Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 4032.63it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [7]:
record_counts = {}
with open(pdb_path, 'r') as f:
    for line in f:
        record_type = line[0:6].strip()
        if record_type:
            record_counts[record_type] = record_counts.get(record_type, 0) + 1

print("--- 文件记录类型统计 ---")
for record, count in record_counts.items():
    print(f"{record:<10s}: {count} 行")

if 'HETATM' in record_counts:
    print("\n[结论] -> 文件中包含 HETATM 记录，需要清理！")
else:
    print("\n[结论] -> 文件中不含 HETATM 记录。")

--- 文件记录类型统计 ---
HEADER    : 1 行
COMPND    : 1 行
ATOM      : 205 行
END       : 1 行

[结论] -> 文件中不含 HETATM 记录。


In [14]:
# PDB ATOM/HETATM 记录的列定义
# 参考: https://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM
colspecs = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26), 
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78), (78, 80)]
names = ['record_name', 'atom_serial', 'atom_name', 'alt_loc', 'res_name', 'chain_id', 
         'res_seq', 'i_code', 'x', 'y', 'z', 'occupancy', 'temp_factor', 'element', 'charge']

# 读取PDB文件
pdb_df = pd.read_fwf(pdb_path, colspecs=colspecs, names=names, header=None)

# 过滤掉空的或非原子记录的行
pdb_df = pdb_df[pdb_df['record_name'].isin(['ATOM', 'HETATM'])].reset_index(drop=True)

print("--- PDB文件前5行预览 ---")
display(pdb_df.head())

print("\n--- 不同残基/分子名称统计 ---")
# .value_counts() 会统计每个名称出现的次数
print(pdb_df['res_name'].value_counts())

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/rna01/liuzhiyuan/zyliu/nai/NExT-Mol/data/cross_docked_new/crossdocked_pocket/ALDR_HUMAN_1_316_0/4qr6_A_rec_4qbx_30l_lig_tt_min_0_pocket10.pdb  '

In [12]:
import pickle
import random
from pathlib import Path

# --- 请设置为您的数据集根目录 ---
dataset_root = Path("/mnt/rna01/liuzhiyuan/zyliu/nai/NExT-Mol/data/cross_docked_new/crossdocked_pocket") 

# 1. 构建主索引文件的路径
index_path = dataset_root / "index.pkl"

if not index_path.exists():
    print(f"错误：主索引文件未找到！请检查路径：{index_path}")
else:
    print(f"成功找到主索引文件：{index_path}")

    # 2. 读取主索引文件 (index.pkl)
    with open(index_path, 'rb') as f:
        master_index = pickle.load(f)

    # 3. 从主索引中提取所有唯一的口袋PDB文件相对路径
    all_pocket_paths_relative = sorted(list(set([item[0] for item in master_index if item[0] is not None])))
    
    print(f"从索引中成功加载了 {len(all_pocket_paths_relative)} 个唯一的口袋文件路径。")

    # 4. 随机抽样并进行检查
    num_samples_to_check = 30 # 可以适当增加抽样数量以获得更全面的了解
    if len(all_pocket_paths_relative) > num_samples_to_check:
        sample_relative_paths = random.sample(all_pocket_paths_relative, num_samples_to_check)
    else:
        sample_relative_paths = all_pocket_paths_relative

    print(f"\n随机抽查 {len(sample_relative_paths)} 个PDB文件...\n" + "="*80)

    # --- 新增计数器 ---
    found_hetatm_count = 0
    found_alt_loc_count = 0
    clean_count = 0

    for relative_path in sample_relative_paths:
        full_pdb_path = dataset_root / relative_path
        
        if not full_pdb_path.exists():
            print(f"文件: {relative_path} -> 状态: 警告 - 文件不存在！")
            continue

        # --- 新增标志位 ---
        has_hetatm = False
        has_alt_loc = False
        
        with open(full_pdb_path, 'r', errors='ignore') as f:
            for line in f:
                # 检查 HETATM
                if line.startswith("HETATM"):
                    has_hetatm = True
                
                # 检查 ALT LOC (在ATOM记录的第17列，即index 16)
                elif line.startswith("ATOM") and line[16] != ' ':
                    has_alt_loc = True
                
                # 如果两种情况都找到了，可以提前结束对该文件的读取
                if has_hetatm and has_alt_loc:
                    break
        
        # --- 动态生成状态报告 ---
        statuses = []
        if has_hetatm:
            statuses.append("🔴 发现 HETATM")
        if has_alt_loc:
            statuses.append("🟡 发现 ALT LOC")
        
        if not statuses:
            statuses.append("✅ 纯净")
            clean_count += 1
        
        # --- 更新计数器 ---
        if has_hetatm: found_hetatm_count += 1
        if has_alt_loc: found_alt_loc_count += 1
            
        final_status = ", ".join(statuses)
        print(f"文件: {relative_path:<70} -> 状态: {final_status}")

    print("="*80 + "\n抽查完毕。")
    print(f"[总结] -> 在抽查的 {len(sample_relative_paths)} 个样本中: ")
    print(f"  - {found_hetatm_count} 个文件包含 HETATM 记录。")
    print(f"  - {found_alt_loc_count} 个文件包含 ALT LOC 记录。")
    print(f"  - {clean_count} 个文件两种问题都未发现。")

    if found_hetatm_count > 0 or found_alt_loc_count > 0:
        print("\n[结论] -> 数据集中存在需要处理的文件。为了保证数据一致性和模型输入的稳定性，")
        print("         执行一个统一的、能同时处理HETATM和ALT LOC的预处理流程是必要的。")
    else:
        print("\n[结论] -> 本次抽查中未发现 HETATM 或 ALT LOC 记录。您的数据集质量非常高。")
        print("         尽管如此，为了保证流程的鲁棒性，仍然推荐在处理所有文件时都运行清理函数。")

成功找到主索引文件：/mnt/rna01/liuzhiyuan/zyliu/nai/NExT-Mol/data/cross_docked_new/crossdocked_pocket/index.pkl
从索引中成功加载了 166500 个唯一的口袋文件路径。

随机抽查 30 个PDB文件...
文件: CHK2_HUMAN_210_526_0/2wtd_A_rec_2cn8_dbq_lig_tt_docked_0_pocket10.pdb  -> 状态: ✅ 纯净
文件: MDM2_HUMAN_16_114_0/5trf_D_rec_4oq3_2v8_lig_tt_docked_0_pocket10.pdb   -> 状态: ✅ 纯净
文件: CARP_CRYPA_90_419_catalytic_0/3wz8_A_rec_5hct_61p_lig_tt_min_0_pocket10.pdb -> 状态: 🟡 发现 ALT LOC
文件: POL_HV1B1_826_919_0/5fdl_A_rec_5vqs_9kd_lig_tt_min_0_pocket10.pdb      -> 状态: ✅ 纯净
文件: EED_HUMAN_76_441_0/5h17_A_rec_5u5k_7vy_lig_tt_docked_0_pocket10.pdb    -> 状态: ✅ 纯净
文件: KAPCA_HUMAN_8_351_0/2uzt_A_rec_3zo2_15i_lig_tt_docked_1_pocket10.pdb   -> 状态: ✅ 纯净
文件: HYES_HUMAN_230_551_0/3wk6_A_rec_5akl_6n8_lig_tt_min_0_pocket10.pdb     -> 状态: ✅ 纯净
文件: ADH1E_HORSE_2_375_0/1axg_A_rec_2jhg_nad_lig_tt_docked_0_pocket10.pdb   -> 状态: ✅ 纯净
文件: BRD9_HUMAN_133_241_0/4uiu_A_rec_5igm_bmf_lig_tt_docked_1_pocket10.pdb  -> 状态: ✅ 纯净
文件: PIM1_HUMAN_121_402_0/5toe_A_rec_5tel_7aj_lig_tt_mi